## Steerable MinMaxScaler

In [ ]:
import pandas as pd
import numpy as np
import tempfile as tf
import os
from progressivis import Scheduler, Print
from progressivis.io import SimpleCSVLoader, DynVar
from progressivis.stats import Histogram2D, Min, Max
from progressivis.datasets import get_dataset
from progressivis.vis import Heatmap
from progressivis.table.constant import Constant
from progressivis.utils.psdict import PsDict
from progressivis.stats.scaling import MinMaxScaler
from progressivis.core.utils import gather_and_run
from progressivis_nb_widgets.nbwidgets import IScaler
from progressivis.datasets import get_dataset

s = Scheduler.default = Scheduler()
cols = ['A', 'B']
csv = SimpleCSVLoader(get_dataset('bigfile_multiscale'), usecols=cols, throttle=100)
dvar = DynVar({'delta': -5, 'ignore_max':10})
sc = MinMaxScaler(reset_threshold=10_000)
sc.create_dependent_modules(csv)
sc.control = dvar
sc.input.control = dvar.output.result
pr=Print(proc=lambda x: None, scheduler=s)
pr.input[0] = sc.output.info
histogram2d = Histogram2D('A', # compute a 2d histogram
                          'B', 
                          xbins=256, ybins=256)
histogram2d.input.table = sc.output.result
histogram2d.input.min = sc.min.output.result
histogram2d.input.max = sc.max.output.result
heatmap=Heatmap() # compute the Heatmap
heatmap.input.array = histogram2d.output.result

**NB:** the results will appear below after running all cells :

In [ ]:
import ipywidgets as ipw
from IPython.display import display
wg = None
async def _after_run(m, run_number):
    global wg
    img = m.get_image_bin()
    if img is None:
        return
    if wg is None:
        wg = ipw.Image(value=img, width=512, height=512)
        display(wg)
    else:
        wg.value = img
heatmap.after_run_proc = _after_run
isc = IScaler(sc)
display(isc)
gather_and_run(s.start(), *isc.coroutines)

